In [ ]:
'''
Initial code was copied from:
https://github.com/jason71995/Keras-GAN-Library
'''

In [12]:
import tensorflow as tf
import keras.backend as K
from keras.models import Sequential
from keras.layers import Conv2D,GlobalAveragePooling2D,LeakyReLU,Conv2DTranspose,Activation,BatchNormalization
from keras.optimizers import Adam

from keras.layers import Dense
from keras import initializers
from keras.layers.core import Dropout



noise_dim = 5

def build_generator(input_shape):

    generator = Sequential()
    
    generator.add(Dense(512,
                        input_dim = noise_dim))
    generator.add(BatchNormalization())
    generator.add(LeakyReLU(0.2))
    
    generator.add(Dense(512))
    generator.add(BatchNormalization())
    generator.add(LeakyReLU(0.2))
    

    


    
    generator.add(Dense(105, activation='tanh'))
    return generator


def build_discriminator(input_shape):

    discriminator = Sequential()
    discriminator.add(Dense(512,
                            input_dim=105))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))
    
    
    discriminator.add(Dense(512))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))
    

        

    
    discriminator.add(Dense(1, activation='sigmoid'))

    return discriminator

def build_functions(batch_size, noise_size, image_size, generator, discriminator):

    noise = K.random_normal((batch_size,) + noise_size,0.0,1.0,"float32")
    real_image = K.placeholder((batch_size,) + image_size)

    fake_image = generator(noise)

    d_input = K.concatenate([real_image, fake_image], axis=0)
    pred_real, pred_fake = tf.split(discriminator(d_input), num_or_size_splits = 2, axis = 0)

    pred_real = K.clip(pred_real,K.epsilon(),1-K.epsilon())
    pred_fake = K.clip(pred_fake,K.epsilon(),1-K.epsilon())

    d_loss = -(K.mean(K.log(pred_real)) + K.mean(K.log(1-pred_fake)))
    g_loss = -K.mean(K.log(pred_fake))

    # get updates of mean and variance in batch normalization layers
    d_updates = discriminator.get_updates_for([d_input])
    g_updates = generator.get_updates_for([noise])

    d_training_updates = Adam(lr=0.0001, beta_1=0.0, beta_2=0.9).get_updates(d_loss, discriminator.trainable_weights)
    d_train = K.function([real_image, K.learning_phase()], [d_loss],d_updates + d_training_updates)

    g_training_updates = Adam(lr=0.0001, beta_1=0.0, beta_2=0.9).get_updates(g_loss, generator.trainable_weights)
    g_train = K.function([real_image, K.learning_phase()], [g_loss], g_updates + g_training_updates)

    return d_train,g_train

In [3]:
# Loading MHAD data for action1, all persons and all repeatations of each person
from utils.data_loader import data_loader
data_object= data_loader(matlab_action_path='../gan/')
myData, mymin, mymax = data_object.actions_normalised([1], twoD_true_or_threeD_false=False)
myData.shape

(117562, 105)

In [13]:
import keras

#from gan_libs.DCGAN import build_generator, build_discriminator, build_functions
# from gan_libs.LSGAN import build_generator, build_discriminator, build_functions
# from gan_libs.SNGAN import build_generator, build_discriminator, build_functions
# from gan_libs.WGAN_GP import build_generator, build_discriminator, build_functions

from utils.common import set_gpu_config, predict_images
from utils.draw_pose import draw_pose
import numpy as np

set_gpu_config("0",0.5)

epoch = 200 + 1
steps = 1000
image_size = (1,1,105)
noise_size = (1,1,5)
batch_size = 16

x_train = myData

generator = build_generator(noise_size)
#print(generator.summary())
discriminator = build_discriminator(image_size)
#print(discriminator.summary())
d_train, g_train = build_functions(batch_size, noise_size, image_size, generator, discriminator)

import time
start_time = time.time()

for e in range(epoch):
    for s in range(steps):
        real_images = x_train[np.random.permutation(x_train.shape[0])[:batch_size]]
        real_images.shape = (batch_size,1,1,105)
        d_loss, = d_train([real_images, 1])
        g_loss, = g_train([real_images, 1])
     
    
    print ("[{0}/{1}] d_loss: {2:.4}, g_loss: {3:.4}".format(e, epoch, d_loss, g_loss))
    #generating a sample
    image = generator.predict(np.zeros(shape=(1,5)))
    image = np.array(image)
    draw_pose(image.reshape(105),'output',"e{0}".format(e))
        
    if e % 100 == 0:
        generator.save_weights("e{0}_generator.h5".format(e))
        discriminator.save_weights("e{0}_discriminator.h5".format(e))
        
elapsed_time = time.time() - start_time 
print('\n\n\n\nNumber of parameter for the Generator and discriminator respectively:\n')
print(generator.count_params())
print('')
print(discriminator.count_params())
print('\n\nNumber of Epochs and steps for each epoch:\n')
print('epochs: ',epoch, '   Steps: ', steps)

print('\n\nTime Taken:')
print(time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))



[0/201] d_loss: 0.301, g_loss: 2.864
[1/201] d_loss: 0.2996, g_loss: 5.112
[2/201] d_loss: 0.2718, g_loss: 3.648
[3/201] d_loss: 1.4, g_loss: 2.018
[4/201] d_loss: 0.8151, g_loss: 6.701
[5/201] d_loss: 0.5833, g_loss: 3.155
[6/201] d_loss: 0.5462, g_loss: 3.231
[7/201] d_loss: 0.5413, g_loss: 2.609
[8/201] d_loss: 0.8006, g_loss: 3.513
[9/201] d_loss: 1.024, g_loss: 1.285
[10/201] d_loss: 0.7238, g_loss: 1.191
[11/201] d_loss: 1.041, g_loss: 2.429
[12/201] d_loss: 1.501, g_loss: 0.4243
[13/201] d_loss: 0.6743, g_loss: 2.499
[14/201] d_loss: 0.7943, g_loss: 1.553
[15/201] d_loss: 0.8254, g_loss: 1.241
[16/201] d_loss: 0.7171, g_loss: 1.615
[17/201] d_loss: 0.6396, g_loss: 5.085
[18/201] d_loss: 1.064, g_loss: 3.669
[19/201] d_loss: 0.5153, g_loss: 4.441
[20/201] d_loss: 0.8033, g_loss: 6.648
[21/201] d_loss: 0.4746, g_loss: 4.208
[22/201] d_loss: 0.4766, g_loss: 2.16
[23/201] d_loss: 2.016, g_loss: 2.28
[24/201] d_loss: 0.4096, g_loss: 2.541
[25/201] d_loss: 1.097, g_loss: 3.167
[26/201

In [ ]:
generator.load_weights("e400_generator.h5".format(e))
discriminator.load_weights("e400_discriminator.h5".format(e))

In [ ]:
for i in range(100):
    x= np.array((i/100,0,0,0,0)).reshape(1,5)
    image = generator.predict(x)
    image = np.array(image)
    draw_pose(image.reshape(105),'output',"e{0}".format(i))